In [20]:


import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from bs4 import BeautifulSoup          # BeautifulSoup is a useful library for extracting data from HTML and XML documents
from numpy import array
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Activation, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, GlobalMaxPooling1D, Dense, Embedding, LSTM, GRU ,SpatialDropout1D
import pandas.testing as tm
from tensorflow import keras
from tensorflow.keras import layers


[nltk_data] Downloading package stopwords to /home/abbas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/abbas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Setting the variable :
numWord = 1000 
maxlen = 200  
batch_size = 32
dataPath= "kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv"
testDataPath = "Data/Sentiment_test.csv"
usecol = ['text' , 'sentiment']

In [3]:
df = pd.read_csv(dataPath, usecols=usecol)
df = df[df.sentiment != "Neutral"]
test_samples = df.sample(n=100, random_state=42)
df = df.drop(test_samples.index)
test_samples.to_csv(testDataPath, index=False)

In [4]:
df.sentiment.value_counts()

Negative    8419
Positive    2210
Name: sentiment, dtype: int64

In [5]:
df.shape

(10629, 2)

In [6]:
df.head(10)

,sentiment,text
1,Positive,RT @ScottWalker: Didn't catch the full #GOPdeb...
3,Positive,RT @RobGeorge: That Carly Fiorina is trending ...
4,Positive,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...
5,Positive,"RT @GregAbbott_TX: @TedCruz: ""On my first day ..."
6,Negative,RT @warriorwoman91: I liked her and was happy ...
8,Negative,Deer in the headlights RT @lizzwinstead: Ben C...
9,Negative,RT @NancyOsborne180: Last night's debate prove...
10,Negative,@JGreenDC @realDonaldTrump In all fairness #Bi...
11,Positive,RT @WayneDupreeShow: Just woke up to tweet thi...
12,Negative,Me reading my family's comments about how grea...


In [7]:
def Preprocess(df):

# removing the html strips
    def strip_html(text):
        # BeautifulSoup is a useful library for extracting data from HTML and XML documents
        soup = BeautifulSoup(text, "html.parser")
        return soup.get_text()


    def remove_punctuations(text):

        pattern = r'[^a-zA-Z0-9\s]'
        text = re.sub(pattern,'',text)

        text = re.sub(r"\s+[a-zA-Z]\s+", ' ', text)
        text = re.sub(r'\s+', ' ', text)

        return text




    stopword_list = nltk.corpus.stopwords.words('english')
#     print(stopword_list)

    updated_stopword_list = []

    for word in stopword_list:
        if word=='not' or word.endswith("n't"):
            pass
        else:
            updated_stopword_list.append(word)

    


    # removing the stopwords
    def remove_stopwords(text, is_lower_case=False):
        # splitting strings into tokens (list of words)
        tokens = nltk.tokenize.word_tokenize(text)
        tokens = [token.strip() for token in tokens]
        if is_lower_case:
            # filtering out the stop words
            filtered_tokens = [token for token in tokens if token not in updated_stopword_list]
        else:
            filtered_tokens = [token for token in tokens if token.lower() not in updated_stopword_list]
        filtered_text = ' '.join(filtered_tokens)
        return filtered_text

    df=df.replace("RT" , '')
    df = strip_html(df)
    df = remove_punctuations(df)
    df = remove_stopwords(df)
    return df


In [8]:
df['text'] = df['text'].apply(Preprocess)
df['sentiment'] = df['sentiment'].apply(lambda x : 1 if x == "Positive" else 0)

In [9]:
df.sentiment.value_counts()

0    8419
1    2210
Name: sentiment, dtype: int64

In [10]:
Y = df['sentiment'].values  # Already 1 or 0 from your lambda function
# Y = pd.get_dummies(df['sentiment']).values
# Step 3: Train-test split
xTrain, xTest, yTrain, yTest = train_test_split(df['text'], Y, test_size=0.2, random_state=42)


In [36]:
# yTrain

array([0, 0, 0, ..., 0, 0, 0])

In [12]:
# # Dataset loading 
# import pandas as pd

# # splits = {'train': 'train_df.csv', 'validation': 'val_df.csv', 'test': 'test_df.csv'}
# train = pd.read_csv(dataPath + splits["train"], nrows=5000)
# validation = pd.read_csv(dataPath + splits["validation"], nrows=1500)
# test = pd.read_csv(dataPath + splits["test"])
# test.to_parquet("Data/testingData.parquet")
# del test
# xTrain = train['text']
# xTest = validation['text']
# yTrain = train['label']
# yTest = validation['label']

In [13]:
# yTrain.value_counts() , xTrain.shape ,train.isnull().sum()

In [14]:
# xTrain = Preprocess(xTrain)
# xTest = Preprocess(xTest)


In [11]:
tokenizer = Tokenizer(num_words=numWord)
tokenizer.fit_on_texts(xTrain)

X_train_tok = tokenizer.texts_to_sequences(xTrain)
X_test_tok = tokenizer.texts_to_sequences(xTest)

In [12]:
# Find the vocabulary size and perform padding on both train and test set
vocab_size = len(tokenizer.word_index) + 1

maxlen = 1000

X_train_pad = pad_sequences(X_train_tok, padding='post', maxlen=maxlen, truncating='post')
X_test_pad = pad_sequences(X_test_tok, padding='post', maxlen=maxlen, truncating='post')

In [13]:
maxlen

1000

In [14]:

# EMBEDDING_DIM = 128

# print('Build model...')

# model = Sequential()
# model.add(Embedding(input_dim = 2000, output_dim = EMBEDDING_DIM, input_shape=(maxlen,) ))
# model.add(LSTM(units=40,  dropout=0.2, recurrent_dropout=0.2))
# model.add(Dense(1, activation='sigmoid'))

# # Try using different optimizers and different optimizer configs
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# print('Summary of the built model...')
# print(model.summary())


# embed_dim = 100
# lstm_out = 120
# max_fatures =1000
# model = Sequential()
# model.add(Embedding(max_fatures, embed_dim,input_shape=(maxlen,)))
# model.add(SpatialDropout1D(0.3))
# model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
# # model.add(Dense(1, activation='sigmoid'))
# model.add(Dense(2,activation='softmax'))
# model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
# print(model.summary())
# from tensorflow import keras

embed_dim = 32  # Embedding dimension

lstm_model = keras.Sequential([
    layers.Embedding(vocab_size, embed_dim, input_shape=(maxlen,)),
    layers.LSTM(64, return_sequences=False), # Using 64 LSTM units
    layers.Dense(64, activation="relu"),
    layers.Dropout(0.5), # Dropout for regularization
    layers.Dense(1, activation="sigmoid") # Output layer for binary classification (positive/negative)
])
lstm_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
lstm_model.summary()

/home/abbas/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
2025-05-03 13:40:32.771956: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 1000, 32)       │       433,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        24,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 462,145 (1.76 MB)

 Trainable params: 462,145 (1.76 MB)

 Non-trainable params: 0 (0.00 B)

In [41]:
# yTrain

In [15]:
c = 0
for i in yTest:
    if i == 0:
        c +=1

In [16]:
c/len(yTest)

0.7911571025399812

In [17]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
class_weight = {0: 1.0, 1: 3.0}

history = lstm_model.fit(X_train_pad, yTrain, batch_size=256, epochs=5, class_weight = class_weight,validation_split=0.2, callbacks=[early_stop])

Epoch 1/5
27/27 ━━━━━━━━━━━━━━━━━━━━ 19s 649ms/step - accuracy: 0.7024 - loss: 0.9655 - val_accuracy: 0.7831 - val_loss: 0.6444
Epoch 2/5
27/27 ━━━━━━━━━━━━━━━━━━━━ 17s 632ms/step - accuracy: 0.7999 - loss: 0.9583 - val_accuracy: 0.7831 - val_loss: 0.6351
Epoch 3/5
27/27 ━━━━━━━━━━━━━━━━━━━━ 17s 624ms/step - accuracy: 0.7926 - loss: 0.9719 - val_accuracy: 0.7831 - val_loss: 0.6295
Epoch 4/5
27/27 ━━━━━━━━━━━━━━━━━━━━ 22s 817ms/step - accuracy: 0.7929 - loss: 0.9664 - val_accuracy: 0.7831 - val_loss: 0.6437
Epoch 5/5
27/27 ━━━━━━━━━━━━━━━━━━━━ 22s 802ms/step - accuracy: 0.7888 - loss: 0.9723 - val_accuracy: 0.7831 - val_loss: 0.6185


In [18]:
lstm_model.save('lstm_model.keras')

In [19]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
from sklearn.metrics import classification_report, accuracy_score


# === Load test data ===
test_df = pd.read_csv(testDataPath)

# === Preprocess test data ===
X_test_texts = test_df['text'].astype(str).tolist()
X_test_seq = tokenizer.texts_to_sequences(X_test_texts)
X_test_pad = pad_sequences(X_test_seq, maxlen=100)  # Adjust maxlen to match training

# === Encode labels ===
label_map = {'Negative': 0, 'Positive': 1}
y_test = test_df['sentiment'].map(label_map).values

# === Load trained model ===
# model = load_model("lstm_model.keras")

# === Inference ===
y_pred_probs = lstm_model.predict(X_test_pad)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()

# === Evaluation ===
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=['Negative', 'Positive']))


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Accuracy: 0.74

Classification Report:
               precision    recall  f1-score   support

    Negative       0.74      1.00      0.85        74
    Positive       0.00      0.00      0.00        26

    accuracy                           0.74       100
   macro avg       0.37      0.50      0.43       100
weighted avg       0.55      0.74      0.63       100



/home/abbas/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/abbas/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/abbas/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [55]:
import tensorflow as tf

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1, **kwargs):
        super().__init__(**kwargs)
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

# Define Token and Position Embedding layer
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

print("\nBuilding Transformer Model...")
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x) # Pool the sequence output
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(1, activation="sigmoid")(x) # Output layer

transformer_model = keras.Model(inputs=inputs, outputs=outputs)
transformer_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
transformer_model.summary()

# --- 5. Training ---
epochs = 3 # Keep epochs low for demonstration purposes

epochs = 3 
print("\n--- Training Transformer Model ---")
history_transformer = transformer_model.fit(
    xTrain, yTrain,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(xTest, yTest)
)

# --- 6. Evaluation ---
print("\n--- Evaluating Models ---")

loss_transformer, accuracy_transformer = transformer_model.evaluate(x_test, y_test, verbose=0)

print(f"Transformer Model Test Accuracy: {accuracy_transformer:.4f}")

# --- 7. Inference Pipeline ---



Building Transformer Model...


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 1000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ token_and_position_embedding_3  │ (None, 1000, 32)       │       468,480 │
│ (TokenAndPositionEmbedding)     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block               │ (None, 1000, 32)       │        10,656 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 32)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 20)             │           660 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 20)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 479,817 (1.83 MB)

 Trainable params: 479,817 (1.83 MB)

 Non-trainable params: 0 (0.00 B)


--- Training Transformer Model ---


ValueError: Invalid dtype: object

In [ ]:
print("\n--- Inference Pipeline ---")

# Get the word index from the IMDB dataset
word_index = keras.datasets.imdb.get_word_index()
# Create reverse word index (integer to word)
reverse_word_index = {v: k for k, v in word_index.items()}

# Function to decode reviews (integers to text) - mainly for understanding
def decode_review(encoded_review):
    # Indices are offset by 3 because 0 = padding, 1 = start, 2 = unknown
    return " ".join([reverse_word_index.get(i - 3, "?") for i in encoded_review])

# Function to preprocess user text for inference
def preprocess_text(text):
    # Lowercase and remove punctuation
    text = text.lower()
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)
    # Split into words
    words = text.split()
    # Convert words to integers using the word_index
    # Use index 2 for OOV (out-of-vocabulary) words
    encoded = [word_index.get(word, 2) + 3 for word in words] # Add 3 for offset
    # Pad the sequence
    padded = keras.preprocessing.sequence.pad_sequences([encoded], maxlen=maxlen)
    return padded

# Function to predict sentiment
def predict_sentiment(text, model):
    processed_text = preprocess_text(text)
    prediction = model.predict(processed_text, verbose=0)[0][0] # Get the probability score
    # Score represents the probability of being positive (class 1)
    return {"negative_score": 1 - prediction, "positive_score": prediction}

# Example Usage
print("\nTesting Inference Pipeline:")
test_sentence_1 = "This movie was fantastic! The acting was superb and the plot was gripping."
test_sentence_2 = "What a waste of time. The plot was predictable and the characters were boring."

print(f"\nSentence 1: '{test_sentence_1}'")
scores_lstm_1 = predict_sentiment(test_sentence_1, lstm_model)
scores_transformer_1 = predict_sentiment(test_sentence_1, transformer_model)
print(f"  LSTM Scores: Negative={scores_lstm_1['negative_score']:.4f}, Positive={scores_lstm_1['positive_score']:.4f}")
print(f"  Transformer Scores: Negative={scores_transformer_1['negative_score']:.4f}, Positive={scores_transformer_1['positive_score']:.4f}")

print(f"\nSentence 2: '{test_sentence_2}'")
scores_lstm_2 = predict_sentiment(test_sentence_2, lstm_model)
scores_transformer_2 = predict_sentiment(test_sentence_2, transformer_model)
print(f"  LSTM Scores: Negative={scores_lstm_2['negative_score']:.4f}, Positive={scores_lstm_2['positive_score']:.4f}")
print(f"  Transformer Scores: Negative={scores_transformer_2['negative_score']:.4f}, Positive={scores_transformer_2['positive_score']:.4f}")

# --- Optional: Save Models ---
# lstm_model.save("lstm_sentiment_model.keras")
# transformer_model.save("transformer_sentiment_model.keras")
# print("\nModels saved.")